In [12]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [14]:
(x_train, y_train),(x_test,y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

In [3]:
# CNN -> BatchNorm -> Relu (Comon structure)
class CNNBlock(layers.Layer):
    def __init__(self, out_channels, kernel_size =3):
        super(CNNBlock, self).__init__()
        self.conv = layers.Conv2D(out_channels, kernel_size, padding ='same')
        self.bn = layers.BatchNormalization()

    def call(self, input_tensor, training = False):
        x = self.conv(input_tensor)
        x= self.bn(x,training= training)
        x= tf.nn.relu(x)
        return x

In [5]:
class ResBlock(layers.Layer):
    def __init__(self,channels):
        super(ResBlock,self).__init__ ()
        self.cnn1= CNNBlock(channels[0])
        self.cnn2= CNNBlock(channels[1])
        self.cnn3= CNNBlock(channels[2])
        self.pooling = layers.MaxPooling2D()
        self.identity_mapping = layers.Conv2D(channels[1],3,padding ='same')

    def call(self, input_tensor, training = False):
        x= self.cnn1(input_tensor,training= training)
        x= self.cnn2(x, training = training)
        x= self.cnn3(
            x + self.identity_mapping(input_tensor), training = training
        )
        return self.pooling(x)
    

class Resnet_Like(keras.Model):
    def __init__(self,num_classes = 10):
        super(Resnet_Like,self).__init__()
        self.block1 = ResBlock([32,32,64])
        self.block2 = ResBlock([128,128,256])
        self.block3 = ResBlock([128,256,512])
        self.pool = layers.GlobalAveragePooling2D()
        self.classifier = layers.Dense(num_classes)
    def call(self, input_tensor, training = False):
        x = self.block1(input_tensor,training = training)
        x = self.block2(x,training = training)
        x = self.block3(x,training = training)
        x= self.pool(x)
        return self.classifier(x)

    


In [6]:
# model = keras.Sequential(
#     [
#         CNNBlock(32),
#         CNNBlock(64),
#         CNNBlock(128),
#         layers.Flatten(),
#         layers.Dense(20), 
#     ]
# )
model = Resnet_Like(num_classes=10)

In [7]:
model.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    metrics =["accuracy"],
    run_eagerly=True,
)
# print(model.summary())

In [9]:
model.fit(x_train, y_train, batch_size=64, epochs=5, verbose=2)
print(model.summary())
model.evaluate(x_test,y_test,batch_size=64, verbose=2)

Epoch 1/5
938/938 - 76s - loss: 0.0395 - accuracy: 0.9879 - 76s/epoch - 81ms/step
Epoch 2/5
938/938 - 76s - loss: 0.0304 - accuracy: 0.9905 - 76s/epoch - 81ms/step
Epoch 3/5
938/938 - 76s - loss: 0.0251 - accuracy: 0.9920 - 76s/epoch - 81ms/step
Epoch 4/5
938/938 - 90s - loss: 0.0232 - accuracy: 0.9928 - 90s/epoch - 96ms/step
Epoch 5/5
938/938 - 101s - loss: 0.0211 - accuracy: 0.9929 - 101s/epoch - 108ms/step
Model: "resnet__like"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 res_block (ResBlock)        multiple                  28896     
                                                                 
 res_block_1 (ResBlock)      multiple                  592512    
                                                                 
 res_block_2 (ResBlock)      multiple                  2364032   
                                                                 
 global_average_pooling2d (G  multipl

[0.05276459455490112, 0.9840999841690063]